# **파이토치 기본**

In [ ]:
import torch

In [ ]:
x= 3.5
y= x*x +2
print(x,y)

3.5 14.25


In [ ]:
x=torch.tensor(3.5)
y=x+3
print(x,y)

tensor(3.5000) tensor(6.5000)


In [ ]:
x=torch.tensor(3.5,requires_grad=True)
y=(x-1)*(x-2)*(x-3)
print(x,y)

tensor(3.5000, requires_grad=True) tensor(1.8750, grad_fn=<MulBackward0>)


In [ ]:
y.backward()
x.grad

tensor(5.7500)

In [ ]:
x=torch.tensor(3.5,requires_grad=True)
y=x*x
z=2*y+3

In [ ]:
z.backward()
x.grad

tensor(14.)

In [ ]:
a=torch.tensor(2.0,requires_grad=True)
b=torch.tensor(1.0,requires_grad=True)

x=2*a + 3*b
y=5*a*a+3*b*b*b
z=2*x+3*y

In [ ]:
z.backward()
a.grad , b.grad

(tensor(64.), tensor(33.))

# **파이토치로 만드는 첫 번째 신경망**

In [ ]:
import torch
import pandas
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import Dataset

In [ ]:
from google.colab import drive
drive.mount('./mount')

Mounted at ./mount


In [ ]:
df=pandas.read_csv('mount/My Drive/Colab Notebooks/gan_basic/mnist_train.csv',header=None)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
row = 13
data = df.iloc[row]
label = data[0]
img = data[1:].values.reshape(28,28)
plt.title("label =" +str(label))
plt.imshow(img,interpolation='none',cmap='binary')
plt.show()

In [ ]:
class Classifier(nn.Module):
  def __init__(self):
    super().__init__()

    self.model = nn.Sequential(
        nn.Linear(784,200),
        nn.Sigmoid(),
        nn.Linear(200,10),
        nn.Sigmoid()
    )

    self.loss_function = nn.MSELoss()

    self.optimiser = torch.optim.SGD(self.parameters(),lr=0.01)
    
    self.counter=0
    self.progress =[]
    
    pass

  def forward(self, inputs):
    return self.model(inputs)

  def train(self,inputs,targets):
    outputs = self.forward(inputs)
    loss = self.loss_function(outputs,targets)
   
    
    self.counter +=1
    if(self.counter %10 ==0):
      self.progress.append(loss.item())
      pass
    if(self.counter %10000 ==0):
      print("counter=",self.counter)
      pass

    self.optimiser.zero_grad()
    loss.backward()
    self.optimiser.step()
    
    pass

  def plot_progress(self):
    df = pandas.DataFrame(self.progress,columns=['loss'])
    df.plot(ylim=(0,1.0),figsize=(16,8),alpha=0.1,marker='.',grid=True,yticks=(0,0.25,0.5))
    pass


  pass


In [ ]:
class MnistDataset(Dataset):
  def __init__(self,csv_file):
    self.data_df = pandas.read_csv(csv_file,header=None)
    pass
  def __len__(self):
    return len(self.data_df)
  def __getitem__(self,index):
    label =self.data_df.iloc[index,0]
    target = torch.zeros((10))
    target[label]=1.0

    image_values= torch.FloatTensor(self.data_df.iloc[index,1:].values)/255.0
    return label,image_values,target

  def plot_image(self,index):
    img = self.data_df.iloc[index,1:].values.reshape(28,28)
    plt.title("label = "+ str(self.data_df.iloc[index,0]))
    plt.imshow(img,interpolation='none',cmap='binary')
    pass
  pass



In [ ]:
mnist_dataset=MnistDataset('mount/My Drive/Colab Notebooks/gan_basic/mnist_train.csv')

In [ ]:
mnist_dataset.plot_image(9)

In [ ]:
mnist_dataset[100]

In [ ]:
C=Classifier()

In [ ]:
%%time

epochs = 3

for i in range(epochs):
  print('training epoch', i+1, "of", epochs)
  for label,image_data_tensor,target_tensor in mnist_dataset:
    C.train(image_data_tensor,target_tensor)
    pass
  pass

In [ ]:
C.plot_progress()

In [ ]:
mnist_test_dataset=MnistDataset('mount/My Drive/Colab Notebooks/gan_basic/mnist_test.csv')

In [ ]:
record =19
mnist_test_dataset.plot_image(record)

In [ ]:
score = 0
items =0
image_data = mnist_test_dataset[record][1]
output =C.forward(image_data)
pandas.DataFrame(output.detach().numpy()).plot(kind='bar',legend=False,ylim=(0,1))


In [ ]:
score = 0
items = 0
for label,image_data_tensor,target_tensor in mnist_test_dataset:
  answer = C.forward(image_data_tensor).detach().numpy()
  if(answer.argmax()==label):
    score +=1
    pass
  items +=1

  pass

print(score,items,score/items)


8640 10000 0.864


# **성능 향상 기법**

In [ ]:
class Classifier(nn.Module):
  def __init__(self):
    super().__init__()

    self.model = nn.Sequential(
        nn.Linear(784,200),
        nn.LeakyReLU(0.02),
        nn.LayerNorm(200),
        nn.Linear(200,10),
        nn.Sigmoid()
    )

    self.loss_function = nn.BCELoss()

    self.optimiser = torch.optim.Adam(self.parameters())
    
    self.counter=0
    self.progress =[]
    
    pass

  def forward(self, inputs):
    return self.model(inputs)

  def train(self,inputs,targets):
    outputs = self.forward(inputs)
    loss = self.loss_function(outputs,targets)
   
    
    self.counter +=1
    if(self.counter %10 ==0):
      self.progress.append(loss.item())
      pass
    if(self.counter %10000 ==0):
      print("counter=",self.counter)
      pass

    self.optimiser.zero_grad()
    loss.backward()
    self.optimiser.step()
    
    pass

  def plot_progress(self):
    df = pandas.DataFrame(self.progress,columns=['loss'])
    df.plot(ylim=(0,1.0),figsize=(16,8),alpha=0.1,marker='.',grid=True,yticks=(0,0.25,0.5))
    pass


  pass


In [ ]:
C=Classifier()

In [ ]:
%%time

epochs = 3

for i in range(epochs):
  print('training epoch', i+1, "of", epochs)
  for label,image_data_tensor,target_tensor in mnist_dataset:
    C.train(image_data_tensor,target_tensor)
    pass
  pass

In [ ]:
C.plot_progress()

In [ ]:
score = 0
items =0
image_data = mnist_test_dataset[record][1]
output =C.forward(image_data)
pandas.DataFrame(output.detach().numpy()).plot(kind='bar',legend=False,ylim=(0,1))
